Bibliotecas

In [ ]:
import pandas as pd
import numpy as np

from scipy import stats
from scipy.stats import zscore
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo, FactorAnalyzer
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

Importando dataset numérico

In [ ]:
df = pd.read_csv("../databases/ENEM_2023_FINAL_num.csv")
display(df)

ETAPA 2 - PLANEJAMENTO DA ANÁLISE FATORIAL (Segundo o livro do Hair)

Selecionando variáveis métricas

In [ ]:
var = ['NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'EST_IDADE', 'EST_RENDA_PER_CAP','EST_CELULAR_PER_CAP', 'EST_COMP_PER_CAP', 'EST_VEICULO_PER_CAP', 'EST_ELE_DOM_PER_CAP']

df_fatores = df[var].dropna()

ETAPA 3 - TESTE DE SUPOSIÇÕES (Segundo o livro do Hair)

Matriz de correlação

In [ ]:
plt.figure(figsize=(7,5))
sns.heatmap(df_fatores.corr(), annot=True, cmap='coolwarm')
plt.title('Matriz de Correlação')
plt.show()

Teste de Esfericidade de Bartlett

In [ ]:
chi_square_value, p_value = calculate_bartlett_sphericity(df_fatores)
print(f"Chi² Bartlett: {chi_square_value:.3f}, p-valor: {p_value:.5f}")

# Se p-valor < 0,05 -> há correlação significativas, ou seja, dá para aplicar a técnica

Medida de adequação da amostra (KMO - geral e MSA - cada var)

In [ ]:
kmo_all, kmo_model = calculate_kmo(df_fatores)
print(f"KMO global: {kmo_model:.3f}")

msa_df = pd.DataFrame({'Variável': var, 'MSA': kmo_all})
print("\nMedida de Adequação Amostral (MSA) por variável:")
print(msa_df)

# KMO global >= 0,80 (ótimo)

# MSA individual se < 0,5 (nenhuma precisa ser excluída)

Matriz anti-imagem: observar se as correlações parciais são baixas

In [ ]:
# O módulo factor_analyzer não gera a matriz anti-imagem diretamente.
# Mas podemos obtê-la a partir da inversa da matriz de correlação:
corr_matrix = df_fatores.corr()
inv_corr = np.linalg.inv(corr_matrix)
anti_image = -inv_corr / np.sqrt(np.outer(np.diag(inv_corr), np.diag(inv_corr)))

# A diagonal da anti-imagem é a MSA
anti_image_diag = pd.Series(np.diag(anti_image), index=corr_matrix.columns)
print("\nMatriz Anti-imagem (diagonal = MSA):")
display(pd.DataFrame(anti_image.round(3)))

# Conferir se as correlações parciais (fora da diagonal) são baixas
plt.figure(figsize=(9,7))
sns.heatmap(anti_image, annot=True, cmap='coolwarm')
plt.title('Matriz Anti-imagem')
plt.show()

Verificar normalidade

In [ ]:
for col in df_fatores.columns:
    plt.figure(figsize=(4,4))
    sns.histplot(df_fatores[col], kde=True)
    plt.title(f'Distribuição - {col}')
    plt.show()

# --- Assimetria e Curtose ---
normalidade = pd.DataFrame({
    'Variável': df_fatores.columns,
    'Assimetria': df_fatores.skew(),
    'Curtose': df_fatores.kurtosis()
})
print(normalidade)

# --- Teste de normalidade (Kolmogorov-Smirnov) ---
# (opcional, já que a amostra é grande)
for col in df_fatores.columns:
    ks_stat, p = stats.kstest(
        (df_fatores[col] - df_fatores[col].mean()) / df_fatores[col].std(), 'norm'
    )
    print(f'{col}: p-valor = {p:.5f}')

Verificar linearidade 

In [ ]:
sns.pairplot(df_fatores, diag_kind='kde', corner=True)
plt.suptitle("Verificação da Linearidade entre as Variáveis", y=1.02)
plt.show()

Verificar homoscedasticidade 

In [ ]:
plt.figure(figsize=(6,3))
sns.boxplot(data=df_fatores[['NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']])
plt.title('Verificação visual da homoscedasticidade - notas')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(6,3))
sns.boxplot(data=df_fatores[['EST_CELULAR_PER_CAP', 'EST_COMP_PER_CAP', 'EST_VEICULO_PER_CAP', 'EST_ELE_DOM_PER_CAP']])
plt.title('Verificação visual da homoscedasticidade - per captas')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(3,3))
sns.boxplot(data=df_fatores[['EST_IDADE']])
plt.title('Verificação visual da homoscedasticidade - idade')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(3,3))
sns.boxplot(data=df_fatores[['EST_RENDA_PER_CAP']])
plt.title('Verificação visual da homoscedasticidade - renda')
plt.xticks(rotation=45)
plt.show()

ETAPA 4 - EXTRAÇÃO DE FATORES (Segundo o livro do Hair)

Padronização dos dados

In [ ]:
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_fatores), columns=df_fatores.columns)

Extração de fatores - PCA

In [ ]:
# --- PCA usando FactorAnalyzer ---
pca = FactorAnalyzer(n_factors=df_scaled.shape[1], method='principal', rotation=None)
pca.fit(df_scaled)

# --- Autovalores (Eigenvalues) ---
ev, v = pca.get_eigenvalues()

# Exibir autovalores
for i, val in enumerate(ev, 1):
    print(f"Componente {i}: autovalor = {val:.3f}")

# --- Scree Plot ---
plt.figure(figsize=(6,3))
plt.scatter(range(1, df_scaled.shape[1]+1), ev)
plt.plot(range(1, df_scaled.shape[1]+1), ev)
plt.title('Scree Plot - PCA')
plt.xlabel('Fator')
plt.ylabel('Autovalor')
plt.axhline(1, color='red', linestyle='--')
plt.show()

# Variância explicada
var_exp = pd.DataFrame({
    'Fator': np.arange(1, len(ev)+1),
    'Autovalor': ev,
    'Variância (%)': (ev / sum(ev)) * 100,
    'Variância acumulada (%)': np.cumsum((ev / sum(ev)) * 100)
})
display(var_exp)

Extração de fatores - FA

In [ ]:
# --- FA (Factor Analysis) ---
fa = FactorAnalyzer(n_factors=3, method='minres', rotation=None)
fa.fit(df_scaled)

# --- Autovalores e variância explicada (FA) ---
ev_fa, v_fa = fa.get_eigenvalues()
print(f"Autovalores FA: {ev_fa}")

# Variância explicada
fa_var = pd.DataFrame({
    'Fator': np.arange(1, len(ev_fa)+1),
    'Autovalor': ev_fa,
    'Variância (%)': (ev_fa / sum(ev_fa)) * 100,
    'Variância acumulada (%)': np.cumsum((ev_fa / sum(ev_fa)) * 100)
})
display(fa_var)

# Cargas fatoriais
fa = FactorAnalyzer(n_factors=3, method='minres', rotation=None)
fa.fit(df_scaled)

loadings = pd.DataFrame(fa.loadings_, index=df_scaled.columns, columns=[f'Fator {i+1}' for i in range(3)])
display(loadings)

Comparar PCA x FA (Autovalores + Scree Plot)

In [ ]:
plt.figure(figsize=(6,3))
plt.plot(range(1, len(ev)+1), ev, marker='o', label='PCA')
plt.plot(range(1, len(ev_fa)+1), ev_fa, marker='s', label='FA')
plt.axhline(1, color='red', linestyle='--')
plt.title('Comparação Scree Plot - PCA vs FA')
plt.xlabel('Fator')
plt.ylabel('Autovalor')
plt.legend()
plt.show()
# Isso te permite visualizar se ambos métodos sugerem o mesmo número de fatores.

Percentual total de variância explicada

In [ ]:
var_exp_fa = np.sum(fa.get_factor_variance()[1]) * 100
print(f"Variância total explicada pelos fatores: {var_exp_fa:.2f}%")

# O ideal era >= 60%

ETAPA 5 - ROTAÇÃO DOS FATORES (Segundo o livro do Hair)

Rotação ortogonal (Varimax)

In [ ]:
# --- Análise fatorial com rotação Varimax ---
fa_varimax = FactorAnalyzer(n_factors=3, method='minres', rotation='varimax')
fa_varimax.fit(df_scaled)

# --- Cargas fatoriais ---
loadings_varimax = pd.DataFrame(
    fa_varimax.loadings_,
    index=df_scaled.columns,
    columns=[f'Fator {i+1}' for i in range(3)]
)

print("Matriz fatorial rotacionada - Varimax:")
display(loadings_varimax.round(3))

# --- Comunalidades ---
comunalidades_varimax = pd.Series(fa_varimax.get_communalities(), index=df_scaled.columns)
print("\nComunalidades (Varimax):")
display(comunalidades_varimax.round(3))

# > 0,70         A variável é muito bem explicada pelos fatores.
# 0,50 – 0,69    Explicação moderada, aceitável.
# 0,30 – 0,49    Explicação baixa, variável não se ajusta bem ao modelo.
# < 0,30         A variável não é bem representada → pode ser candidata à exclusão.

Rotação oblíqua (Promax)

In [ ]:
# --- Análise fatorial com rotação Promax ---
fa_promax = FactorAnalyzer(n_factors=3, method='minres', rotation='promax')
fa_promax.fit(df_scaled)

# --- Cargas fatoriais ---
loadings_promax = pd.DataFrame(
    fa_promax.loadings_,
    index=df_scaled.columns,
    columns=[f'Fator {i+1}' for i in range(3)]
)

print("Matriz fatorial rotacionada - Promax:")
display(loadings_promax.round(3))

# --- Comunalidades ---
comunalidades_promax = pd.Series(fa_promax.get_communalities(), index=df_scaled.columns)
print("\nComunalidades (Promax):")
display(comunalidades_promax.round(3))

# Correlação entre fatores (somente na Promax)
corr_fatores = pd.DataFrame(
    fa_promax.phi_,
    index=[f'Fator {i+1}' for i in range(3)],
    columns=[f'Fator {i+1}' for i in range(3)]
)
print("Correlação entre os fatores (Promax):")
display(corr_fatores.round(3))


Maiores cargas das rotações para melhor visuzalização

In [ ]:
# --- Mostrar as maiores cargas (Varimax) ---
loadings_varimax_high = loadings_varimax.applymap(lambda x: x if abs(x) >= 0.4 else '')
print("Cargas fatoriais significativas (≥ 0,4) - Varimax:")
display(loadings_varimax_high)

# --- Mostrar as maiores cargas (Promax) ---
loadings_promax_high = loadings_promax.applymap(lambda x: x if abs(x) >= 0.4 else '')
print("Cargas fatoriais significativas (≥ 0,4) - Promax:")
display(loadings_promax_high)

Gráficos para facilitar a visualização das comunalidades de Varimax e Promax

In [ ]:
# --- Criar DataFrame com comunalidades das duas rotações ---
comunalidades_df = pd.DataFrame({
    'Varimax': comunalidades_varimax,
    'Promax': comunalidades_promax
})

# --- Ordenar por Varimax (para deixar o gráfico mais organizado) ---
comunalidades_df = comunalidades_df.sort_values('Varimax', ascending=False)

# --- Gráfico de barras lado a lado ---
x = np.arange(len(comunalidades_df))
width = 0.35

plt.figure(figsize=(10,6))
plt.bar(x - width/2, comunalidades_df['Varimax'], width, label='Varimax')
plt.bar(x + width/2, comunalidades_df['Promax'], width, label='Promax')

plt.axhline(0.5, color='red', linestyle='--', label='Limite aceitável (0,5)')
plt.title('Comunalidades por variável - Comparação Varimax x Promax')
plt.ylabel('Comunalidade')
plt.xticks(x, comunalidades_df.index, rotation=45, ha='right')
plt.ylim(0, 1)
plt.legend()
plt.tight_layout()
plt.show()

Falta o gráfico com rotações

In [ ]:
# 'fa_promax' é o modelo final, temos 3 fatores
loadings = pd.DataFrame(fa_promax.loadings_, 
                        index=df_scaled.columns, 
                        columns=['Fator 1', 'Fator 2', 'Fator 3'])

# Plotar Fator 1 x Fator 2
plt.figure(figsize=(7,7))
plt.scatter(loadings['Fator 1'], loadings['Fator 2'], color='blue')

# Adicionar rótulos das variáveis
for i, var in enumerate(loadings.index):
    plt.text(loadings['Fator 1'][i]+0.02, loadings['Fator 2'][i], var, fontsize=9)

plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlabel('Fator 1 (Desempenho Acadêmico)')
plt.ylabel('Fator 2 (Condição Socioeconômica)')
plt.title('Mapa Fatorial — Cargas Fatoriais Rotacionadas (Promax)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

ETAPA 6 - INTERPRETANDO OS DADOS (Segundo o livro do Hair - Vide relatório para mais informações)

ETAPA 7 - VALIDAÇÃO (Segundo o livro do Hair)

Calcular escores fatoriais

In [ ]:
# --- Calcular escores fatoriais (usando Promax) ---
fatores_scores = fa_promax.transform(df_scaled)

# --- Criar DataFrame com escores ---
fatores_df = pd.DataFrame(
    fatores_scores,
    columns=[f'Fator {i+1}' for i in range(fa_promax.n_factors)]
)

print("Escores fatoriais (primeiras linhas):")
display(fatores_df.head())

Calcular o Alfa de Cronbach (Confiabilidade Interna)

In [ ]:
def cronbach_alpha(df):
    df_standardized = df.apply(zscore)
    item_variances = df_standardized.var(axis=0, ddof=1)
    total_variance = df_standardized.sum(axis=1).var(ddof=1)
    n_items = df_standardized.shape[1]
    alpha = (n_items / (n_items - 1)) * (1 - item_variances.sum() / total_variance)
    return alpha

fator1_vars = ['NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']  # Desempenho
fator2_vars = ['EST_RENDA_PER_CAP','EST_CELULAR_PER_CAP', 'EST_COMP_PER_CAP', 'EST_VEICULO_PER_CAP', 'EST_ELE_DOM_PER_CAP']  # Socioeconômico
fator3_vars = ['EST_IDADE']  # Perfil individual

# --- Alfa de Cronbach para cada fator ---
alpha_f1 = cronbach_alpha(df_scaled[fator1_vars])
alpha_f2 = cronbach_alpha(df_scaled[fator2_vars])
# fator3 tem apenas uma variável → alfa não é aplicável

print(f"Alfa de Cronbach - Fator 1 (Desempenho Acadêmico): {alpha_f1:.3f}")
print(f"Alfa de Cronbach - Fator 2 (Condição Socioeconômica): {alpha_f2:.3f}")
print("Fator 3 (Idade) contém apenas uma variável, portanto o alfa não se aplica.")
